In [1]:
from astropy.coordinates.angle_utilities import angular_separation
from astropy.coordinates import SkyCoord, get_sun,FK4
from astropy.time import Time

from datetime import timedelta
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

import os
import glob
import numpy as np
import astropy.units as u

from collections import Counter

In [2]:
def bandToFreq(band):
    # Convert the input to a numpy array (if it's not already)
    band = np.asarray(band)
    
    # Create a mapping of band numbers to frequency values
    band_to_freq = {
        1: 0.45, 2: 0.70, 3: 0.90, 4: 1.31,
        5: 2.20, 6: 3.93, 7: 4.70, 8: 6.55, 9: 9.18
    }

    # Use np.vectorize to apply the mapping to each element in the array
    freq = np.vectorize(lambda b: band_to_freq.get(b, -1))(band)
    
    return freq

In [3]:
def sigmaClip(y, N):
    y = np.asarray(y)
    overall_mask = np.ones(len(y), dtype=bool)  # Initialize overall mask to all True
    
    for _ in range(N):
        median = np.median(y[overall_mask])  # Calculate median only on filtered values
        std_dev = np.std(y[overall_mask])    # Calculate std_dev only on filtered values
        
        # Mask values within 1 sigma of the mean
        current_mask = (y >= median - std_dev) & (y <= median + std_dev)
        overall_mask &= current_mask  # Update overall mask with new filtering
    
    return overall_mask

In [4]:
# Load the file to inspect its contents
filename = r"C:\Users\adamf\Documents\PhD\Diffraction\interpolatedRAE2MasterFile.csv"

rawData = pd.read_csv(filename)
data = rawData
data['time'] = pd.to_datetime(data['time'])
data.drop('Unnamed: 0',axis=1,inplace=True)
data.set_index('time',inplace=True)

In [5]:
data.iloc[-100000]

frequency_band         6.000000e+00
position_x            -6.983776e+02
position_y             2.938816e+02
position_z             2.666922e+03
earth_unit_vector_x    6.585006e-01
earth_unit_vector_y    6.826953e-01
earth_unit_vector_z    3.167080e-01
right_ascension        8.294251e+00
declination            7.935045e+01
rv1_coarse             7.116068e+05
rv2_coarse             1.226222e+06
rv1_fine               1.644602e+06
rv2_fine               1.261825e+06
rv_temp                1.092427e+09
Name: 1975-06-20 11:46:15.040000, dtype: float64

In [60]:
start_date = pd.to_datetime("1974-01-01 00:00")
end_date = pd.to_datetime("1975-06-30 23:00")

# Selecting rows within the date range
earthOccult = data[(data.index >= start_date) & (data.index <= end_date)].copy()

In [61]:
earthOccult.head(50)

,frequency_band,position_x,position_y,position_z,earth_unit_vector_x,earth_unit_vector_y,earth_unit_vector_z,right_ascension,declination,rv1_coarse,rv2_coarse,rv1_fine,rv2_fine,rv_temp
time,,,,,,,,,,,,,,
1974-01-01 00:00:16.633,5,-2342.408478,0.726133,1525.056671,-0.965828,-0.188089,-0.178323,11.288027,34.320398,1.375491e+06,4.931740e+06,0.00,0.000000e+00,1092637163
1974-01-01 00:00:18.558,5,-2341.700745,-1.483316,1526.104797,-0.965826,-0.188094,-0.178325,11.291459,34.350330,1.343475e+06,4.346525e+06,0.00,0.000000e+00,1092637163
1974-01-01 00:00:20.483,5,-2340.993011,-3.692764,1527.152924,-0.965825,-0.188098,-0.178327,11.294891,34.380262,1.448626e+06,4.629902e+06,0.00,0.000000e+00,1092637163
1974-01-01 00:00:22.408,5,-2340.285278,-5.902213,1528.201050,-0.965824,-0.188103,-0.178329,11.298324,34.410194,1.343475e+06,4.346525e+06,1380232.00,4.174212e+06,1092637163
1974-01-01 00:00:24.333,5,-2339.577545,-8.111661,1529.249176,-0.965823,-0.188107,-0.178331,11.301756,34.440126,1.312207e+06,4.629902e+06,0.00,0.000000e+00,1092637163
1974-01-01 00:00:26.258,5,-2338.869812,-10.321110,1530.297302,-0.965822,-0.188112,-0.178332,11.305188,34.470058,1.410481e+06,4.095690e+06,0.00,0.000000e+00,1092637163
1974-01-01 00:00:28.183,5,-2338.162079,-12.530558,1531.345428,-0.965820,-0.188116,-0.178334,11.308620,34.499990,1.281664e+06,4.931740e+06,0.00,0.000000e+00,1092637163
1974-01-01 00:00:31.992,4,-2336.746521,-16.949742,1533.441772,-0.965818,-0.188125,-0.178338,11.315486,34.559856,1.519378e+06,7.840331e+05,0.00,0.000000e+00,1092637163
1974-01-01 00:00:33.917,4,-2336.038696,-19.159478,1534.489990,-0.965817,-0.188130,-0.178340,11.318918,34.589790,1.519378e+06,8.339306e+05,0.00,0.000000e+00,1092637163


In [62]:
frequencies = np.sort(earthOccult['frequency_band'].unique())

In [63]:
earthOccult['TimeSince'] = (earthOccult.index-earthOccult.index[0]).total_seconds()

In [26]:
%matplotlib qt
fig, axes = plt.subplots(3, 3, figsize=(9, 5 * len(frequencies)), sharex=True)
for ax, freq in zip(axes.flatten(), frequencies):
    subset = earthOccult[earthOccult['frequency_band']==freq]
    x = subset['TimeSince']
    
    
    
    mask = sigmaClip(subset['rv1_coarse'],2)
    ax.scatter(x,subset['rv1_coarse'])
    #ax.scatter(x,clip,color = 'green')
    
    ax.set_title(f'Frequency: {bandToFreq(freq)}MHz')
    ax.set_xlabel('Time since start of period (s)')
    ax.set_ylabel('rv1_coarse')
    ax.grid(True)

In [27]:
#If a burst occurs in 3 frequency bins, accept it and record it as an event
#run sigma clips on smaller samples and then look at if dots are significantly above
#assign z-score to specific points
#evaluate if there are multiple points above a threshold near each other (all same frequency)
#determine a background variation in the smooth area, and then see if consecutive points are 5 background variation above
#give everything a burst probability score

In [28]:
#check for 7 points in a row above baseline by at least 5 background variations above, in 3 bins within a 30 min window.

In [29]:
def combine_overlapping_windows(windows):
    """
    Combine overlapping or adjacent windows into a single window.

    Parameters:
        windows (list of tuples): List of (start_idx, end_idx) windows.

    Returns:
        list of tuples: Combined windows.
    """
    if not windows:
        return []

    # Sort windows by their start index
    windows = sorted(windows, key=lambda x: x[0])

    # Initialize with the first window
    combined_windows = [windows[0]]

    for start, end in windows[1:]:
        last_start, last_end = combined_windows[-1]

        # Check for overlap or adjacency
        if start <= last_end:  # Overlapping or adjacent windows
            # Merge windows
            combined_windows[-1] = (last_start, max(last_end, end))
        else:
            # No overlap, add as a new window
            combined_windows.append((start, end))

    return combined_windows

In [30]:
def burstWindows(data, window_time=10, consec_points=7):
    """
    Detect burst windows in the data and return their start and end times as datetime objects.

    Parameters:
        data (pd.DataFrame): The input data with a datetime index.
        window_time (int): Size of the sliding window (in indices).
        consec_points (int): Number of consecutive points exceeding the threshold.

    Returns:
        list of tuples: Start and end times of valid windows as datetime objects.
    """
    freqs = data['frequency_band'].unique()
    freq_indices = {}

    # Detect bursts for each frequency band
    for freq in freqs:
        freq_df = data[data['frequency_band'] == freq]
        background_points = sigmaClip(freq_df['rv1_coarse'], 3)
        med = np.median(freq_df['rv1_coarse'][background_points])
        std = np.std(freq_df['rv1_coarse'][background_points])
        z = (freq_df['rv1_coarse'] - med) / std
        cond = z > 10
        consecutive_points = np.convolve(cond, np.ones(consec_points, dtype=int), mode='valid')
        indices = np.where(consecutive_points == consec_points)[0]
        freq_indices[freq] = indices
        
    # Initialize a list for windows meeting the condition
    valid_windows = []
    # Check for overlap in windows
    max_index = len(data)
    for start_idx in range(max_index - window_time + 1):
        end_idx = start_idx + window_time
        count = 0

        for freq, indices in freq_indices.items():
            # Check if any indices fall within the current window
            in_window = (indices >= start_idx) & (indices < end_idx)
            if np.any(in_window):
                count += 1
            if count >= 3:
                # Store the start and end indices of the window
                valid_windows.append((start_idx, end_idx))
                break
    # Combine overlapping windows
    valid_windows = combine_overlapping_windows(valid_windows)
    # Refine the window times using all frequency bands
    datetime_windows = []
    for start_idx, end_idx in valid_windows:
        # Initialize variables to track the earliest start time and latest end time
        earliest_time = data.index[-1]
        latest_time = data.index[0]
        # Check across all frequency bands to find the earliest start and latest end
        for freq in freqs:
            freq_df = data[data['frequency_band'] == freq]
            try:#this is because sometimes the freq_df are shorter than others, due to gaps in the data
                early_time = freq_df.index[start_idx]
            
                if(early_time<earliest_time):
                    earliest_time = early_time
                late_time = freq_df.index[end_idx]
                if(late_time>latest_time):
                    latest_time = late_time
            except:
                continue
        # Append the refined window times
        if earliest_time and latest_time:
            datetime_windows.append((earliest_time, latest_time))

    return datetime_windows

In [31]:
def burst_detection_chunks(data, window_time=10, consec_points=7, chunk_size='24h'):
    """
    Apply the burstWindows function on chunks of the data and combine overlapping windows.

    Parameters:
        data (pd.DataFrame): The input DataFrame, indexed by datetime objects.
        time_col (str): Name of the column containing datetime objects.
        window_time (int): The size of the sliding window (in indices).
        consec_points (int): Number of consecutive points exceeding the threshold.
        chunk_size (str): The size of each chunk (e.g., 'H' for hourly chunks).

    Returns:
        dict: A dictionary where keys are the start time of each burst (datetime) and values are
              the corresponding (start_time, end_time) for each burst.
    """
    # Ensure the index is sorted by datetime
    data = data.sort_index()

    # Split data into chunks based on chunk_size
    chunks = [
        (start, chunk)
        for start, chunk in data.groupby(pd.Grouper(freq=chunk_size))
    ]
    burst_results = []

    for _, chunk in chunks:
        if not chunk.empty:
            # Apply burstWindows to the current chunk
            valid_windows = burstWindows(chunk, window_time=window_time, consec_points=consec_points)

            
            # Convert indices to datetime values
            for start_time, end_time in valid_windows:
                # Store in the results with the start time of the burst as the key
                burst_results.append(np.array([start_time, end_time]))
    
    return burst_results

In [64]:
bursts = burst_detection_chunks(earthOccult)

In [65]:
print(bursts)

[array([Timestamp('1974-01-01 06:19:06.684000'),
       Timestamp('1974-01-01 06:55:02.754000')], dtype=object), array([Timestamp('1974-01-01 08:56:48.160000'),
       Timestamp('1974-01-01 13:13:32.736000')], dtype=object), array([Timestamp('1974-01-01 17:27:00.630000'),
       Timestamp('1974-01-01 18:31:15.864000')], dtype=object), array([Timestamp('1974-01-01 19:20:40.248000'),
       Timestamp('1974-01-01 20:49:12.682000')], dtype=object), array([Timestamp('1974-01-01 19:56:28.653000'),
       Timestamp('1974-01-01 21:23:59.654000')], dtype=object), array([Timestamp('1974-01-02 01:10:50.104000'),
       Timestamp('1974-01-02 01:27:20.777000')], dtype=object), array([Timestamp('1974-01-02 03:58:55.573000'),
       Timestamp('1974-01-02 04:26:53.617000')], dtype=object), array([Timestamp('1974-01-02 06:24:25.590000'),
       Timestamp('1974-01-02 06:59:52.871000')], dtype=object), array([Timestamp('1974-01-02 10:42:54.794000'),
       Timestamp('1974-01-02 11:23:23.523000')], dtype=

In [72]:
timestamps = [timestamp for burst in bursts for timestamp in burst]

# Extract year and month
year_month = [f"{ts.year}-{ts.month:02d}" for ts in timestamps]

# Count events per month
month_counts = Counter(year_month)

# Convert to a sorted DataFrame for plotting
df = pd.DataFrame(sorted(month_counts.items()), columns=["Month", "Count"])
df["Month"] = pd.to_datetime(df["Month"])

# Plot histogram
plt.figure(figsize=(10, 6))
plt.bar(df["Month"], df["Count"], width=20, align='center')
plt.gca().xaxis_date()
plt.xlabel("Month")
plt.ylabel("Number of Burst Events")
plt.title("Histogram of Burst Events by Month")
plt.grid()
plt.tight_layout()
plt.show()

In [24]:
df.head(20)

,Month,Count
0,1974-01-01,324
1,1974-02-01,306
2,1974-03-01,64
3,1974-05-01,460
4,1974-06-01,408
5,1974-07-01,308
6,1974-08-01,304
7,1974-09-01,576
8,1974-10-01,320
9,1974-11-01,444


In [67]:
monthly_counts = earthOccult.groupby(pd.Grouper(freq='M')).size()

In [68]:
monthly_hours = monthly_counts*1.9/3600

In [69]:
timestamps = [timestamp for burst in bursts for timestamp in burst]
year_month = [f"{ts.year}-{ts.month:02d}" for ts in timestamps]
month_counts = Counter(year_month)

# Step 2: Convert month_counts into a DataFrame
df_counts = pd.DataFrame(sorted(month_counts.items()), columns=["Month", "Count"])
df_counts["Month"] = pd.to_datetime(df_counts["Month"])
df_counts["Month"] = df_counts["Month"] + pd.offsets.MonthEnd(0)  # Shift to month-end

# Step 3: Prepare monthly_counts as DataFrame (already at month-end)
monthly_counts_df = monthly_hours.reset_index()
monthly_counts_df.columns = ["Month", "Total"]

# Step 4: Merge dataframes on Month and fill missing Total values with 0
merged_df = pd.merge(df_counts, monthly_counts_df, on="Month", how="outer").fillna({"Total": 0, "Count": 0})

# Step 5: Calculate rate safely, avoiding division by zero
merged_df["Rate"] = merged_df.apply(lambda row: row["Count"] / row["Total"] if row["Total"] > 0 else 0, axis=1)

# Step 6: Plot the rate
plt.figure(figsize=(10, 6))
plt.bar(merged_df["Month"], merged_df["Rate"], width=20, align="center")
plt.gca().xaxis_date()
plt.xlabel("Month")
plt.ylabel("Rate of Burst Events Per Hour")
plt.title("Rate of Burst Events by Month")
plt.grid()
plt.tight_layout()
plt.show()

In [73]:
# Step 1: Flatten burst timestamps and count occurrences per month
timestamps = [timestamp for burst in bursts for timestamp in burst]
year_month = [f"{ts.year}-{ts.month:02d}" for ts in timestamps]
month_counts = Counter(year_month)

# Step 2: Convert month_counts into a DataFrame
df_counts = pd.DataFrame(sorted(month_counts.items()), columns=["Month", "Count"])
df_counts["Month"] = pd.to_datetime(df_counts["Month"])
df_counts["Month"] = df_counts["Month"] + pd.offsets.MonthEnd(0)  # Shift to month-end

# Step 3: Prepare monthly_counts as DataFrame (already at month-end)
monthly_counts_df = monthly_hours.reset_index()
monthly_counts_df.columns = ["Month", "Total"]

# Step 4: Merge dataframes on Month and fill missing Total values with 0
merged_df = pd.merge(df_counts, monthly_counts_df, on="Month", how="outer").fillna({"Total": 0, "Count": 0})

# Step 5: Calculate rate safely, avoiding division by zero
merged_df["Rate"] = merged_df.apply(lambda row: row["Count"] / row["Total"] if row["Total"] > 0 else 0, axis=1)

# Step 6: Calculate error bars as sqrt of counts
merged_df["Error"] = merged_df["Count"].apply(lambda count: np.sqrt(count) if count > 0 else 0)
merged_df["Month"] = merged_df["Month"] - pd.tseries.offsets.DateOffset(months=1)
# Step 7: Plot the rate with error bars
plt.figure(figsize=(10, 6))
plt.bar(
    merged_df["Month"], 
    merged_df["Rate"], 
    width=20, 
    align="center", 
    label="Rate of Burst Events"
)
plt.errorbar(
    merged_df["Month"], 
    merged_df["Rate"], 
    yerr=merged_df["Error"] / merged_df["Total"],  # Propagate error in rate
    fmt="none", 
    ecolor="red", 
    capsize=3
)

# Finalize the plot
plt.gca().xaxis_date()
plt.xlabel("Month")
plt.ylabel("Rate of Burst Events Per Hour")
plt.title("Rate of Burst Events by Month")
plt.legend()
plt.grid()
plt.tight_layout()
plt.show()

In [34]:
df_counts.head()

,Month,Count
0,1974-01-01,324
1,1974-02-01,306
2,1974-03-01,64
3,1974-05-01,460
4,1974-06-01,408


In [33]:
merged_df.head()

,Month,Count,Total,Rate
0,1974-01-01,324.0,0.0,0.0
1,1974-02-01,306.0,0.0,0.0
2,1974-03-01,64.0,0.0,0.0
3,1974-05-01,460.0,0.0,0.0
4,1974-06-01,408.0,0.0,0.0


In [59]:
first_date = earthOccult.index[0]
date_title = first_date.strftime("%d %B %Y")

# Plot with the overall title
%matplotlib qt
fig, axes = plt.subplots(3, 3, figsize=(9, 5 * len(frequencies)), sharex=True)

# Call burst_detection_chunks to get the bursts for each frequency band
burst_results = burst_detection_chunks(earthOccult)

# Create a list of frequencies (you may already have this variable)
frequencies = np.sort(earthOccult['frequency_band'].unique())
fig.suptitle(f"Burst Detection Sample - {date_title}", fontsize=16)
# Plotting each frequency band's data and detected bursts
for ax, freq in zip(axes.flatten(), frequencies):
    subset = earthOccult[earthOccult['frequency_band'] == freq]
    x = subset['TimeSince']
    y = subset['rv1_coarse']
    
    # Sigma clipping to establish baseline
    mask = sigmaClip(y, 3)
    med = np.median(y[mask])
    std = np.std(y[mask])
    
    # Plot all points
    ax.scatter(x, y, label='Data', alpha=0.5)
    
    # Highlight points in all burst windows
    for burst in burst_results:
        start_time, end_time = burst  # Unpack burst start and end times
        if freq in subset['frequency_band'].values:
            # Convert time indices to match data index
            valid_x = subset[(subset.index >= start_time) & (subset.index <= end_time)]['TimeSince']
            valid_y = subset[(subset.index >= start_time) & (subset.index <= end_time)]['rv1_coarse']
            
            # Plot the burst window in red
            ax.scatter(valid_x, valid_y, color='red', label='Burst Window', alpha=0.7)
    
    # Add 10-sigma line for context
    ax.axhline(med + 10 * std, color='green', linestyle='--', label='10σ Threshold')
    ax.axhline(med, color='blue', linestyle='-', label='Median')
    
    # Set plot details
    ax.set_title(f'Frequency: {bandToFreq(freq)}MHz', fontsize=10)
    
    # Only add x-axis label to the bottom row of plots
    if ax.is_last_row():
        ax.set_xlabel('Time since start of period (s)', fontsize=8)
    
    # Always add y-axis label
    ax.set_ylabel('Upper V (Uncalibrated)', fontsize=8)
    
    ax.grid(True)

# Avoid duplicate labels in the legend
handles, labels = ax.get_legend_handles_labels()
unique = dict(zip(labels, handles))
fig.legend(unique.values(), unique.keys(), loc='upper right')
fig.subplots_adjust(top=0.92, hspace=0.4, wspace=0.4)
plt.show()

In [51]:
%matplotlib qt
fig, axes = plt.subplots(3, 3, figsize=(12, 8), sharex=True, sharey=True)

# Get the combined windows from burst detection results
burst_results = burst_detection_chunks(earthOccult, window_time=10, consec_points=7, chunk_size='24h')

# Get unique frequencies
frequencies = earthOccult['frequency_band'].unique()

for ax, freq in zip(axes.flatten(), frequencies):
    # Subset the data for the current frequency
    subset = earthOccult[earthOccult['frequency_band'] == freq]
    x = subset.index  # Use datetime index directly
    y = subset['rv1_coarse']
    
    # Sigma clipping to establish baseline
    mask = sigmaClip(y, 3)
    med = np.median(y[mask])
    std = np.std(y[mask])
    
    # Plot all points
    ax.scatter(x, y, label='Data', alpha=0.5)
    
    # Highlight points in all burst windows
    for start_time, (window_start, window_end) in burst_results.items():
        if freq in subset['frequency_band'].unique():
            # Find the overlapping indices in the subset
            valid_subset = subset.loc[window_start:window_end]
            valid_x = valid_subset.index  # Use datetime indices
            valid_y = valid_subset['rv1_coarse']
            ax.scatter(valid_x, valid_y, color='red', label='Burst Window', alpha=0.7)
    
    # Add 3-sigma line for context
    ax.axhline(med + 10 * std, color='green', linestyle='--', label='10σ Threshold')
    ax.axhline(med, color='blue', linestyle='-', label='Median')
    
    # Set plot details
    ax.set_title(f'Frequency: {bandToFreq(freq)} MHz')
    ax.set_xlabel('Time')
    ax.set_ylabel('rv1_coarse')
    ax.grid(True)

# Avoid duplicate labels in the legend
handles, labels = ax.get_legend_handles_labels()
unique = dict(zip(labels, handles))
fig.legend(unique.values(), unique.keys(), loc='upper right')
plt.subplots_adjust(hspace=0.8, wspace=0.4)
plt.tight_layout()
plt.show()

AttributeError: 'list' object has no attribute 'items'

In [29]:
len(combined_windows)

1

In [45]:
print(combined_windows[0][0])

86
